In [83]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

In [84]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

# Import Nanophotonics Data

In [108]:
fomsx = torch.Tensor(loadmat('FOMs_x.mat', mat_dtype=True)['RVs_rnd'])
fomsy = torch.Tensor(loadmat('FOMs_y.mat', mat_dtype=True)['FOMs'])

# F0 Fidelity

In [109]:
f0 = torch.cat((fomsx[:,0:5], fomsy[:,0:1]), 1)
f0 = f0[torch.randperm(f0.size()[0])]
data = f0
#if not os.path.isfile('elevators.mat'):
#    print('Downloading \'elevators\' UCI dataset...')
#    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')

#data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

In [110]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 500))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(500, 50))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(50, 2))

feature_extractor = LargeFeatureExtractor()

In [111]:
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2)),
                num_dims=2, grid_size=100
            )
            self.feature_extractor = feature_extractor

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            # We're also scaling the features so that they're nice values
            projected_x = self.feature_extractor(x)
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [112]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [113]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 60
def train():
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

# See dkl_mnist.ipynb for explanation of this flag
with gpytorch.settings.use_toeplitz(True):
    %time train()

Iter 1/60 - Loss: 316070.625
Iter 2/60 - Loss: 309448.188
Iter 3/60 - Loss: 308253.219
Iter 4/60 - Loss: 303485.375
Iter 5/60 - Loss: 299088.781
Iter 6/60 - Loss: 295577.312
Iter 7/60 - Loss: 292145.844
Iter 8/60 - Loss: 288821.594
Iter 9/60 - Loss: 286085.781
Iter 10/60 - Loss: 283105.406
Iter 11/60 - Loss: 279701.938
Iter 12/60 - Loss: 276345.469
Iter 13/60 - Loss: 273538.188
Iter 14/60 - Loss: 270164.844
Iter 15/60 - Loss: 266928.844
Iter 16/60 - Loss: 263319.469
Iter 17/60 - Loss: 260412.109
Iter 18/60 - Loss: 257992.734
Iter 19/60 - Loss: 254647.266
Iter 20/60 - Loss: 251479.234
Iter 21/60 - Loss: 248186.828
Iter 22/60 - Loss: 247364.766
Iter 23/60 - Loss: 250328.406
Iter 24/60 - Loss: 248557.516
Iter 25/60 - Loss: 245838.516
Iter 26/60 - Loss: 242942.141
Iter 27/60 - Loss: 238439.234
Iter 28/60 - Loss: 235690.781
Iter 29/60 - Loss: 232020.172
Iter 30/60 - Loss: 228226.219
Iter 31/60 - Loss: 225700.219
Iter 32/60 - Loss: 222669.406
Iter 33/60 - Loss: 220501.422
Iter 34/60 - Loss: 

In [114]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    preds = model(test_x)

In [116]:
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    train_preds = model(train_x)
print('Train MAE: {}'.format(torch.mean(torch.abs(train_preds.mean - train_y))))

Train MAE: 426.04217529296875


/opt/anaconda3/envs/ML/lib/python3.7/site-packages/gpytorch/models/exact_gp.py:247: UserWarning: The input matches the stored training data. Did you forget to call model.train()?
  "The input matches the stored training data. Did you forget to call model.train()?", UserWarning


In [115]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 534.7948608398438


# F1 Fidelity

In [95]:
f1 = torch.cat((fomsx[:,0:5], fomsy[:,1:2]), 1)
f1 = f1[torch.randperm(f0.size()[0])]
data = f1

X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

In [96]:
data_dim = train_x.size(-1)

feature_extractor = LargeFeatureExtractor()

In [97]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [98]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 60
def train():
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

# See dkl_mnist.ipynb for explanation of this flag
with gpytorch.settings.use_toeplitz(True):
    %time train()

Iter 1/60 - Loss: 78329.734
Iter 2/60 - Loss: 75709.570
Iter 3/60 - Loss: 71425.562
Iter 4/60 - Loss: 69685.180
Iter 5/60 - Loss: 67653.703
Iter 6/60 - Loss: 66153.445
Iter 7/60 - Loss: 64489.352
Iter 8/60 - Loss: 62887.633
Iter 9/60 - Loss: 60827.512
Iter 10/60 - Loss: 59943.648
Iter 11/60 - Loss: 58588.652
Iter 12/60 - Loss: 57837.777
Iter 13/60 - Loss: 55990.848
Iter 14/60 - Loss: 55376.008
Iter 15/60 - Loss: 54486.082
Iter 16/60 - Loss: 52747.555
Iter 17/60 - Loss: 51616.535
Iter 18/60 - Loss: 50745.480
Iter 19/60 - Loss: 49326.977
Iter 20/60 - Loss: 48739.941
Iter 21/60 - Loss: 47984.977
Iter 22/60 - Loss: 46625.273
Iter 23/60 - Loss: 45830.047
Iter 24/60 - Loss: 45367.789
Iter 25/60 - Loss: 45465.441
Iter 26/60 - Loss: 43923.406
Iter 27/60 - Loss: 43364.781
Iter 28/60 - Loss: 42830.148
Iter 29/60 - Loss: 41213.836
Iter 30/60 - Loss: 40647.664
Iter 31/60 - Loss: 40651.199
Iter 32/60 - Loss: 40183.098
Iter 33/60 - Loss: 40771.688
Iter 34/60 - Loss: 39745.715
Iter 35/60 - Loss: 3985

In [99]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    preds = model(test_x)

In [100]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 151.2612762451172


# F2 Fidelity

In [118]:
f2.size()

torch.Size([4983, 6])

In [101]:
f2 = torch.cat((fomsx[:,0:5], fomsy[:,2:3]), 1)
f2 = f2[torch.randperm(f0.size()[0])]
data = f2

X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

In [102]:
data_dim = train_x.size(-1)

feature_extractor = LargeFeatureExtractor()

In [103]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [104]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 60
def train():
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

# See dkl_mnist.ipynb for explanation of this flag
with gpytorch.settings.use_toeplitz(True):
    %time train()

Iter 1/60 - Loss: 140728.109
Iter 2/60 - Loss: 135678.375
Iter 3/60 - Loss: 128216.062
Iter 4/60 - Loss: 125743.219
Iter 5/60 - Loss: 120583.156
Iter 6/60 - Loss: 118685.500
Iter 7/60 - Loss: 115023.516
Iter 8/60 - Loss: 111105.977
Iter 9/60 - Loss: 107162.234
Iter 10/60 - Loss: 103565.539
Iter 11/60 - Loss: 100971.406
Iter 12/60 - Loss: 99462.922
Iter 13/60 - Loss: 97114.367
Iter 14/60 - Loss: 94522.398
Iter 15/60 - Loss: 92956.523
Iter 16/60 - Loss: 91233.789
Iter 17/60 - Loss: 88963.742
Iter 18/60 - Loss: 87190.609
Iter 19/60 - Loss: 85849.875
Iter 20/60 - Loss: 84115.234
Iter 21/60 - Loss: 82339.727
Iter 22/60 - Loss: 81023.695
Iter 23/60 - Loss: 79612.508
Iter 24/60 - Loss: 78161.945
Iter 25/60 - Loss: 76797.984
Iter 26/60 - Loss: 75512.547
Iter 27/60 - Loss: 73915.328
Iter 28/60 - Loss: 72722.336
Iter 29/60 - Loss: 72049.648
Iter 30/60 - Loss: 72479.477
Iter 31/60 - Loss: 72830.672
Iter 32/60 - Loss: 72102.078
Iter 33/60 - Loss: 70202.242
Iter 34/60 - Loss: 68431.594
Iter 35/60 -

In [105]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    preds = model(test_x)

In [106]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 223.6408233642578


# Data Normalization

In [179]:
f2 = torch.cat((fomsx[:,0:5], fomsy[:,2:3]), 1)
f2 = f2[torch.randperm(f0.size()[0])]
data = f2

X_nano = data[:, :-1]
X_nano = X_nano - X_nano.min(0)[0]
X_nano = 2 * (X_nano / X_nano.max(0)[0]) - 1
y_nano = data[:, -1]
y_nano = y_nano - y_nano.min(0)[0]
y_nano = 2 * (y_nano / y_nano.max(0)[0]) - 1

train_n = int(floor(0.8*len(X)))

train_x = X_nano[:train_n, :].contiguous()
train_y = y_nano[:train_n].contiguous()

test_x = X_nano[train_n:, :].contiguous()
test_y = y_nano[train_n:].contiguous()

In [180]:
X.size(), y.size(), train_x.size(), train_y.size(), test_x.size(), test_y.size()

(torch.Size([4983, 5]),
 torch.Size([4983]),
 torch.Size([3986, 5]),
 torch.Size([3986]),
 torch.Size([997, 5]),
 torch.Size([997]))

# Regression with regular GP with squared exponential kernel (RBF)

## Nanophotonics

In [119]:
%load_ext autoreload
%autoreload 2

In [177]:
train_x.size(), train_y.size(), test_x.size(), test_y.size()

(torch.Size([3986, 5]),
 torch.Size([3986]),
 torch.Size([100, 2]),
 torch.Size([997]))

In [159]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=5),
            ), grid_size=grid_size, num_dims=5
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [160]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def train():
    training_iterations = 30
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

%time train()

Iter 1/30 - Loss: 0.854
Iter 2/30 - Loss: 0.826
Iter 3/30 - Loss: 0.797
Iter 4/30 - Loss: 0.770
Iter 5/30 - Loss: 0.743
Iter 6/30 - Loss: 0.715
Iter 7/30 - Loss: 0.689
Iter 8/30 - Loss: 0.662
Iter 9/30 - Loss: 0.637
Iter 10/30 - Loss: 0.613
Iter 11/30 - Loss: 0.590
Iter 12/30 - Loss: 0.569
Iter 13/30 - Loss: 0.547
Iter 14/30 - Loss: 0.530
Iter 15/30 - Loss: 0.513
Iter 16/30 - Loss: 0.499
Iter 17/30 - Loss: 0.492
Iter 18/30 - Loss: 0.479
Iter 19/30 - Loss: 0.476
Iter 20/30 - Loss: 0.470
Iter 21/30 - Loss: 0.469
Iter 22/30 - Loss: 0.464
Iter 23/30 - Loss: 0.469
Iter 24/30 - Loss: 0.471
Iter 25/30 - Loss: 0.472
Iter 26/30 - Loss: 0.477
Iter 27/30 - Loss: 0.484
Iter 28/30 - Loss: 0.484
Iter 29/30 - Loss: 0.485
Iter 30/30 - Loss: 0.482
CPU times: user 40min 8s, sys: 27 s, total: 40min 35s
Wall time: 10min 52s


In [ ]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_train = model(train_x)
    
print('Train MAE: {}'.format(torch.mean(torch.abs(preds_train.mean - train_y))))

model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = model(test_x)
    
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

## Matern Kernel

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(ard_num_dims=5),
            ), grid_size=grid_size, num_dims=5
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model_mat = GPRegressionModel(train_x, train_y, likelihood)

In [ ]:
# Find optimal model hyperparameters
model_mat.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model_mat.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_mat)

def train():
    training_iterations = 50
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model_mat(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

%time train()

In [ ]:
model_mat.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_train = model_mat(train_x)
    
print('Train MAE: {}'.format(torch.mean(torch.abs(preds_train.mean - train_y))))

model_mat.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = model_mat(test_x)
    
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

## Astronomy

### Dataset

In [184]:
from copy import deepcopy
import numpy as np
from scipy.stats import norm
from scipy import integrate

In [186]:
integrateMethod = integrate.trapz
#from snls.SNMFOptFunction import snlsLogLikl
def snlsLogLikl(evalPts, obsData, resolution, numObsToUse):
  """ This computes the log likelihood. """

  # Round them before proceeding
  resolution = int(resolution)
  numObsToUse = int(numObsToUse)

  # Define some constants
  LIGHT_SPEED = 299792.48

  # Prelims
  numObs = obsData.shape[0]
  numPts = evalPts.shape[0]
  if numObsToUse is None:
    numObsToUse = numObs

  # Decompose Data
  useObsData = obsData[0:numObsToUse, :]
  z = useObsData[:, [0]]
  obs = useObsData[:, [1]]
  obsErr = useObsData[:, [2]]

  # Create arrays for storing outputs
  logJointProbs = np.zeros((numPts, 1))
  lumMeans = np.zeros((numPts, numObsToUse))

  # Now iterate through each of the evalPts
  for i in range(numPts):
    H = evalPts[i, 0]
    OmegaM = evalPts[i, 1]
    OmegaL = evalPts[i, 2]
    currLumMeans = np.zeros((numObsToUse, 1))

    f = lambda t: 1/ np.sqrt(OmegaM * (1+t)**3 + OmegaL)
    for obsIter in range(numObsToUse):
      dLC = LIGHT_SPEED * (1 + z[obsIter, 0])/H
      integrate_grid = np.linspace(0, z[obsIter, 0], resolution)
      integrate_grid_res = integrate_grid[1] - integrate_grid[0]
      integrate_values = f(integrate_grid)
      dLI = integrateMethod(integrate_values, dx=integrate_grid_res)
      dL = dLC * dLI
      currLumMeans[obsIter, 0] = 5 * np.log10(dL) + 25
    obsLikl = norm.pdf(obs, currLumMeans, obsErr)
    obsLogLikl = sum(np.log(obsLikl))

    obsLogLikl = obsLogLikl
    logJointProbs[i, 0] = obsLogLikl
    logJointProbs[i, 0] = logJointProbs[i, 0] / numObsToUse
    lumMeans[[i], :] = currLumMeans.transpose()

  return logJointProbs

In [191]:
N = 10000
obsData = np.loadtxt('davisdata.txt')
x1 = np.random.uniform(60,80,(N,))
x2 = np.random.uniform(0.1,1,(N,))
x3 = np.random.uniform(0.1,1,(N,))
x = np.transpose(np.array([x1,x2,x3]))
y = snlsLogLikl(x, obsData, resolution=N, numObsToUse=192)
x, y

(array([[78.50262781,  0.48920912,  0.94813763],
        [63.79143157,  0.14882729,  0.25895187],
        [70.04099213,  0.60479575,  0.48826584],
        ...,
        [63.15587325,  0.87843793,  0.68956331],
        [76.61496609,  0.58598371,  0.43431912],
        [65.20416802,  0.24215812,  0.61851402]]), array([[ -7.76943453],
        [-10.67393402],
        [ -1.8549358 ],
        ...,
        [ -3.72461854],
        [ -3.23301544],
        [ -0.23571951]]))

In [193]:
X_astro = x
X_astro = X_astro - X_astro.min(0)[0]
X_astro = 2 * (X_astro / X_astro.max(0)[0]) - 1
y_astro = y
y_astro = y_astro - y_astro.min(0)[0]
y_astro = 2 * (y_astro / y_astro.max(0)[0]) - 1

train_n = int(floor(0.8*len(X)))

train_x = X_nano[:train_n, :].contiguous()
train_y = y_nano[:train_n].contiguous()

test_x = X_nano[train_n:, :].contiguous()
test_y = y_nano[train_n:].contiguous()

(array([[78.50262781,  0.48920912,  0.94813763],
        [63.79143157,  0.14882729,  0.25895187],
        [70.04099213,  0.60479575,  0.48826584],
        ...,
        [63.15587325,  0.87843793,  0.68956331],
        [76.61496609,  0.58598371,  0.43431912],
        [65.20416802,  0.24215812,  0.61851402]]), array([[ -7.76943453],
        [-10.67393402],
        [ -1.8549358 ],
        ...,
        [ -3.72461854],
        [ -3.23301544],
        [ -0.23571951]]))

In [ ]:
train_x.size(), train_y.size(), test_x.size(), test_y.size()

In [195]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=5),
            ), grid_size=grid_size, num_dims=5
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

In [196]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def train():
    training_iterations = 50
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

%time train()

Iter 1/200 - Loss: 0.854
Iter 2/200 - Loss: 0.826
Iter 3/200 - Loss: 0.797
Iter 4/200 - Loss: 0.769
Iter 5/200 - Loss: 0.741
Iter 6/200 - Loss: 0.715
Iter 7/200 - Loss: 0.689
Iter 8/200 - Loss: 0.662
Iter 9/200 - Loss: 0.636
Iter 10/200 - Loss: 0.614
Iter 11/200 - Loss: 0.590
Iter 12/200 - Loss: 0.568
Iter 13/200 - Loss: 0.550
Iter 14/200 - Loss: 0.530
Iter 15/200 - Loss: 0.516
Iter 16/200 - Loss: 0.501
Iter 17/200 - Loss: 0.491
Iter 18/200 - Loss: 0.481
Iter 19/200 - Loss: 0.473
Iter 20/200 - Loss: 0.473
Iter 21/200 - Loss: 0.468
Iter 22/200 - Loss: 0.469
Iter 23/200 - Loss: 0.469
Iter 24/200 - Loss: 0.474
Iter 25/200 - Loss: 0.478
Iter 26/200 - Loss: 0.473
Iter 27/200 - Loss: 0.486
Iter 28/200 - Loss: 0.484
Iter 29/200 - Loss: 0.487
Iter 30/200 - Loss: 0.479
Iter 31/200 - Loss: 0.489
Iter 32/200 - Loss: 0.493
Iter 33/200 - Loss: 0.493
Iter 34/200 - Loss: 0.479
Iter 35/200 - Loss: 0.494
Iter 36/200 - Loss: 0.480
Iter 37/200 - Loss: 0.508
Iter 38/200 - Loss: 0.481
Iter 39/200 - Loss: 0

In [ ]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_train = model(train_x)
    
print('Train MAE: {}'.format(torch.mean(torch.abs(preds_train.mean - train_y))))

model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = model(test_x)
    
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

In [206]:
train_x, test_x, train_y, test_y, preds_train, preds

(tensor([[-1.0000, -0.6000,  0.8000, -0.3333,  1.0000],
         [-0.1765, -0.8000,  0.6000,  0.6000,  0.4000],
         [ 0.2941,  0.8000,  0.6000,  0.6000,  0.2000],
         ...,
         [-0.7647,  0.8000, -0.5000,  0.4667,  0.6000],
         [-0.0588,  0.8000,  0.0000, -0.3333,  1.0000],
         [-0.5294, -0.6000, -0.4000,  1.0000, -0.4000]]),
 tensor([[ 1.0000,  1.0000,  0.5000, -0.3333,  0.2000],
         [-0.0588, -0.4000, -0.2000, -1.0000,  0.2000],
         [-0.1765,  0.4000, -0.1000, -0.2000,  1.0000],
         ...,
         [-0.7647, -1.0000, -0.4000, -0.8667,  0.6000],
         [ 0.2941,  1.0000,  1.0000,  0.6000, -0.4000],
         [-0.6471,  0.6000,  0.0000,  0.3333,  0.0000]]),
 tensor([-0.6831, -0.5761, -0.5747,  ..., -0.5244, -0.7934, -0.8485]),
 tensor([-0.7902, -0.6887, -0.7329, -0.5410, -0.6914, -0.6122, -0.4607, -0.7129,
         -0.7670, -0.8458, -0.5351, -0.8709, -0.6627, -0.8548, -0.8672,  0.5121,
         -0.6566,  0.3160, -0.5680, -0.7795, -0.6920,  0.5972, 

# Matern Kernel

In [209]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(ard_num_dims=5),
            ), grid_size=grid_size, num_dims=5
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model_mat = GPRegressionModel(train_x, train_y, likelihood)

In [210]:
# Find optimal model hyperparameters
model_mat.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model_mat.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_mat)

def train():
    training_iterations = 50
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model_mat(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

%time train()

Iter 1/50 - Loss: 0.854
Iter 2/50 - Loss: 0.825
Iter 3/50 - Loss: 0.798
Iter 4/50 - Loss: 0.769
Iter 5/50 - Loss: 0.742
Iter 6/50 - Loss: 0.714
Iter 7/50 - Loss: 0.688
Iter 8/50 - Loss: 0.662
Iter 9/50 - Loss: 0.637
Iter 10/50 - Loss: 0.613
Iter 11/50 - Loss: 0.591
Iter 12/50 - Loss: 0.569
Iter 13/50 - Loss: 0.547
Iter 14/50 - Loss: 0.530
Iter 15/50 - Loss: 0.516
Iter 16/50 - Loss: 0.501
Iter 17/50 - Loss: 0.488
Iter 18/50 - Loss: 0.481
Iter 19/50 - Loss: 0.473
Iter 20/50 - Loss: 0.472
Iter 21/50 - Loss: 0.466
Iter 22/50 - Loss: 0.472
Iter 23/50 - Loss: 0.469
Iter 24/50 - Loss: 0.475
Iter 25/50 - Loss: 0.475
Iter 26/50 - Loss: 0.477
Iter 27/50 - Loss: 0.484
Iter 28/50 - Loss: 0.484
Iter 29/50 - Loss: 0.482
Iter 30/50 - Loss: 0.487
Iter 31/50 - Loss: 0.492
Iter 32/50 - Loss: 0.487
Iter 33/50 - Loss: 0.480
Iter 34/50 - Loss: 0.494
Iter 35/50 - Loss: 0.492
Iter 36/50 - Loss: 0.481
Iter 37/50 - Loss: 0.492
Iter 38/50 - Loss: 0.508
Iter 39/50 - Loss: 0.496
Iter 40/50 - Loss: 0.508
Iter 41/5

/opt/anaconda3/envs/ML/lib/python3.7/site-packages/gpytorch/utils/linear_cg.py:296: UserWarning: CG terminated in 1000 iterations with average residual norm 4.756214141845703 which is larger than the tolerance of 1 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  " a gpytorch.settings.max_cg_iterations(value) context.".format(k + 1, residual_norm.mean(), tolerance)


Iter 42/50 - Loss: 0.526
Iter 43/50 - Loss: 0.522
Iter 44/50 - Loss: 0.557
Iter 45/50 - Loss: 0.574
Iter 46/50 - Loss: 0.538
Iter 47/50 - Loss: 0.573
Iter 48/50 - Loss: 0.575
Iter 49/50 - Loss: 0.613
Iter 50/50 - Loss: 0.587
CPU times: user 2h 28min 38s, sys: 1min 39s, total: 2h 30min 18s
Wall time: 28min 36s


In [211]:
model_mat.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_train = model_mat(train_x)
    
print('Train MAE: {}'.format(torch.mean(torch.abs(preds_train.mean - train_y))))

model_mat.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = model_mat(test_x)
    
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

/opt/anaconda3/envs/ML/lib/python3.7/site-packages/gpytorch/utils/linear_cg.py:296: UserWarning: CG terminated in 1000 iterations with average residual norm 1.2756239175796509 which is larger than the tolerance of 0.01 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  " a gpytorch.settings.max_cg_iterations(value) context.".format(k + 1, residual_norm.mean(), tolerance)


Train MAE: 0.3079177737236023
Test MAE: 0.31456151604652405
